#### 原始网页：https://news.qq.com/zt2020/page/feiyan.htm#/
### 1. 数据提取部分

#### 1.1 导入模块

##### 如果使用本地静态资源，需要在头部声明

In [25]:
# # 只需要在顶部声明 CurrentConfig.ONLINE_HOST 即可(针对notebook)
# from pyecharts.globals import CurrentConfig, OnlineHostType
# # OnlineHostType.NOTEBOOK_HOST 默认值为 http://localhost:8888/nbextensions/assets/
# CurrentConfig.ONLINE_HOST = OnlineHostType.NOTEBOOK_HOST

In [26]:
import requests
import json
import time
import datetime
import pandas as pd

In [27]:
today = datetime.datetime.today()
date = f'{today.year}年{today.month}月{today.day}日'
date

'2022年8月21日'

#### 1.2 获取数据

In [28]:
url = "https://api.inews.qq.com/newsqa/v1/query/inner/publish/modules/list?modules=statisGradeCityDetail,diseaseh5Shelf"
response = requests.post(url)
china_datas = response.json()["data"]["diseaseh5Shelf"]["areaTree"][0]["children"]
china_datas[:1]

[{'today': {'local_confirm_add': 22130,
   'abroad_confirm_add': 0,
   'dead_add': 36,
   'confirm': 22130,
   'confirmCuts': 0,
   'isUpdated': True,
   'tip': '',
   'wzz_add': 0},
  'total': {'mtime': '2022-08-21 10:08:35',
   'nowConfirm': 4997545,
   'confirm': 5020895,
   'wzz': 0,
   'highRiskAreaNum': 0,
   'continueDayZeroConfirmAdd': 0,
   'heal': 13742,
   'showHeal': True,
   'provinceLocalConfirm': 0,
   'continueDayZeroConfirm': 0,
   'adcode': '',
   'showRate': False,
   'dead': 9608,
   'mediumRiskAreaNum': 0,
   'continueDayZeroLocalConfirmAdd': 0},
  'children': [{'adcode': '',
    'date': '2022/08/21',
    'today': {'isUpdated': True,
     'wzz_add': '',
     'local_confirm_add': 22130,
     'confirm': 22130,
     'confirmCuts': 0},
    'total': {'dead': 0,
     'mediumRiskAreaNum': 0,
     'adcode': '',
     'showRate': False,
     'provinceLocalConfirm': 0,
     'highRiskAreaNum': 0,
     'continueDayZeroLocalConfirmAdd': 0,
     'mtime': '2022-08-21 10:37:34',
  

#### 1.3 解析提取数据

In [29]:
data_set = []
for data in china_datas:
    data_dict = {
        'province': data['name'], 
        'nowConfirm': data['total']['nowConfirm'], 
        'confirm': data['total']['confirm'], 
        'dead': data['total']['dead'], 
        'heal': data['total']['heal'], 
    } 
    data_set.append(data_dict)

#### 1.4 生成表格对象

In [30]:
df = pd.DataFrame(data_set) 
df

,province,nowConfirm,confirm,dead,heal
0,台湾,4997545,5020895,9608,13742
1,香港,290713,371999,9602,71684
2,海南,7136,7471,6,329
3,广东,370,8701,8,8323
4,西藏,281,285,0,4
5,福建,189,3969,1,3779
6,陕西,164,3554,3,3387
7,天津,159,2116,3,1954
8,上海,139,63762,595,63028
9,吉林,137,40309,5,40167


#### 1.5 数据永久性保存

In [31]:
df.to_csv('./国内疫情数据.csv', encoding='utf-8-sig')

### 2. 可视化操作部分

#### 2.1 导入模块

In [32]:
from pyecharts import options as opts
from pyecharts.charts import Bar, Line, Pie, Grid, Map

#### 2.2 数据整理

In [33]:
df1 = df.sort_values(by=['nowConfirm'], ascending=False)[:10]
df1

,province,nowConfirm,confirm,dead,heal
0,台湾,4997545,5020895,9608,13742
1,香港,290713,371999,9602,71684
2,海南,7136,7471,6,329
3,广东,370,8701,8,8323
4,西藏,281,285,0,4
5,福建,189,3969,1,3779
6,陕西,164,3554,3,3387
7,天津,159,2116,3,1954
8,上海,139,63762,595,63028
9,吉林,137,40309,5,40167


#### 2.3 可视化饼图

##### 提示：可视化前，将数据转换为列表

In [34]:
lst = list(zip(df1['province'].values.tolist(),
           df1['nowConfirm'].values.tolist()))
lst

[('台湾', 4997545),
 ('香港', 290713),
 ('海南', 7136),
 ('广东', 370),
 ('西藏', 281),
 ('福建', 189),
 ('陕西', 164),
 ('天津', 159),
 ('上海', 139),
 ('吉林', 137)]

In [35]:
pie = (
    Pie()
    .add("", lst, radius=["10%", "30%"])
    .set_global_opts(
        title_opts=opts.TitleOpts(title=f'{date}_省份占比'),
        legend_opts=opts.LegendOpts(orient="vertical", pos_top="70%", pos_left="70%"))
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}:{c}")))
pie.render_notebook()

#### 2.4可视化地图

In [36]:
df3 = df.sort_values(by=['nowConfirm'], ascending=False)[0:]
df3 = df3.drop(df3[df3['nowConfirm'] == 0].index)

lst = list(zip(df3['province'].values.tolist(),
           df3['nowConfirm'].values.tolist()))
map = (
    Map()
    .add("", lst, 'china')
    .set_global_opts(
        title_opts=opts.TitleOpts(title=f'{date}_疫情分布'),
        visualmap_opts=opts.VisualMapOpts(
            max_=1000,
            min_=0,
            is_piecewise=False)
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
)
map.render_notebook()
